In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pickle
import os
import copy
from football_sim.all import Calibrator, Season,Settings
%matplotlib inline
import platform

In [2]:
base_dir = './'

settings = Settings(os.path.join(base_dir, 'config.yaml'))

In [ ]:
calib = Calibrator(settings)
calib.download_all_data() 
calib.process_data()

In [ ]:
year = 2020
league = 'BPL'
as_of=pd.to_datetime('2020-01-01')
as_of=None
calib.calibrate_teams(league,year,as_of=as_of)

In [ ]:
season = Season(league, year,calib, use_home_advantage=False,as_of=as_of)
print('home advantage', season.home_advantage)

season.process_current_results()
season.simulate_season(n_scenarios=100000)

season.process_simulation()

In [ ]:

ind=None
#i=season.match_id['bpl_burnley_manchesterunited_2021-01-12']
#hg=season.simulated_home_goals[i,:]
#ag=season.simulated_away_goals[i,:]
#ind=ag>hg
#i=season.match_id['bpl_liverpool_manchesterunited_2021-01-17']
#ind=None
#hg=season.simulated_home_goals[i,:]
#ag=season.simulated_away_goals[i,:]
#ind = ind&(ag>=hg)
#print(100*ind.sum()/ind.size)
# ind= None
#i = season.team_id['Manchester United']
#ind = season.place_per_team[i,:]<=4
ind = None
sr=season.season_report(ind=ind)
sr

In [ ]:
calib.fixtures[0] in calib.fixtures

In [ ]:
plt.plot(sr['rating'],sr['Points (mean)'],'.')

In [ ]:
team_names = list(season.teams.keys())
y=[season.teams[x].offense for x in team_names]
x=[1/season.teams[x].defense for x in team_names]
plt.figure(figsize=(10,10))
#plt.plot(x,y,'.')
c=0
for x_,y_,name in zip(x,y,team_names):
    team = season.teams[name]
    if dict(sr['CL'])[name]>20:
        print(name)
        if c<=9:
            sym='-'
        else:
            sym='-'
        p=plt.plot(1/np.array(team.defense_hist),team.offense_hist,sym,label=name,color='C{:d}'.format(c),linewidth=1)
        c+=1
        plt.plot(x_,y_,'o',color=p[0].get_color())
        name_=''.join([x[:3] for x in name.split()])
        plt.text(x_+0.03,y_,name_,fontsize=9,color=p[0].get_color())
plt.grid(True)
plt.ylabel('off')
plt.xlabel('def')
#plt.legend()

In [ ]:
season.probability_grid(ind=ind);

In [ ]:
season.points_probability_grid(ind=ind);

In [ ]:
my_team = season.teams['Manchester United']

"""
names = []
importance = []
home=[]
away=[]

for m in season.matches_to_sim:
    #p=season.importance(m,ref_team=my_team,place=4,or_better=True)
    names.append(m.id)
    #importance.append(np.sum((p[1:]-p[0])**2))
    importance.append(season.importance(m,ref_team=my_team,place=4,or_better=True))
    home.append(m.home_team_name)
    away.append(m.away_team_name)

names=np.array(names)
importance=np.array(importance)

isort = np.argsort(-importance)
names=names[isort]
importance=importance[isort]
season.team_report(my_team,places=[1,2,3,4])
"""
#matches_to_report = [m for m in season.matches_to_sim if 'manchesterunited' in m.id]
matches_to_report = [m for m in season.matches_to_sim ]

df = pd.DataFrame({'date':[m.date for m in matches_to_report],
                   'id':[m.id for m in matches_to_report],
                   'home':[m.home_team_name for m in matches_to_report],
                   'away':[m.away_team_name for m in matches_to_report],
              'home win':[m.forecast_home_wins for m in matches_to_report],
             'away win':[m.forecast_away_wins for m in matches_to_report],
             'draw':[m.forecast_draw for m in matches_to_report],
            'We Win': [-100+(100+m.forecast_home_wins)*(m.home_team_name ==my_team.name)+(100+m.forecast_away_wins)*(m.away_team_name ==my_team.name) for m in matches_to_report],
            'home goals':[m.forecast_home_goals for m in matches_to_report],
             'away goals':[m.forecast_away_goals for m in matches_to_report],
            'p_cl':[np.round(season.what_if(m,ref_team=my_team,place=4,or_better=True,show_plot=False)[0]).astype(int) for m in matches_to_report],
             'importance':[100*season.importance(m,ref_team=my_team,place=4,or_better=True) for m in matches_to_report]})

#df.sort_values(by='importance',ascending=False)
df['importance']=(df['importance']-df['importance'].min())/(df['importance'].max()-df['importance'].min())
cols = [x for x in df.columns if x not in ['id']]
df[cols].sort_values(by='importance',ascending=False)
df.loc[df['importance']>=0.5,cols]



In [ ]:
n_plot=28

plt.figure(figsize=(16,9))
df2=df.sort_values(by='importance',ascending=False).head(n_plot).copy()
plt.bar(np.arange(n_plot),df2['importance'].values)
plt.xticks(np.arange(n_plot),df2['id'].values,rotation=90);

In [ ]:
i=season.match_id['bpl_liverpool_manchesterunited_2021-01-17']
match = season.matches_to_sim[i]
mufc = season.teams['Manchester United']
#wh = season.teams['West Ham United']
season.what_if(match,ref_team=mufc,place=4,or_better=True)

In [ ]:
match.home_team.outcomes_vs(match.away_team)

In [ ]:
home_team=match.home_team
away_team=match.away_team
if home_team.name == 'Manchester United':
    print('home')
    l_mu=home_team.offense*away_team.defense/90
    l_other=home_team.defense*away_team.offense/90
    other_team_name = away_team.name
elif away_team.name == 'Manchester United':
    print('away')
    l_mu=home_team.defense*away_team.offense/90
    l_other=home_team.offense*away_team.defense/90
    other_team_name = home_team.name
else:
    print('no MU')


g_mu=np.random.exponential(1/l_mu,[100000,100])
g_other=np.random.exponential(1/l_other,[100000,100])
ng_mu=(g_mu.cumsum(axis=1)<90).sum(axis=1)
ng_other=(g_other.cumsum(axis=1)<90).sum(axis=1)



u_res,n=np.unique(np.array([ng_mu,ng_other]),axis=1,return_counts=True)
#isort=np.argsort(-n)
#u_res[:,isort[:10]]
points_for_right_result = 2
points_for_right_score = 3

points=[]

for i in range(u_res.shape[1]):
    guess = u_res[:,i]
    ind_right_score = (ng_mu==guess[0])&((ng_other==guess[1]))
    points_=points_for_right_score*ind_right_score.mean()
    #points_=0
    ind_right_result = (guess[0]>guess[1])&(ng_mu>ng_other)
    ind_right_result = ind_right_result | (guess[0]==guess[1])&(ng_mu==ng_other)
    ind_right_result = ind_right_result | (guess[0]<guess[1])&(ng_mu<ng_other)
    points_+=points_for_right_result*ind_right_result.mean()
    points.append(points_)
    #g_mu[:,0]
    
points=np.array(points)


df=pd.DataFrame({'Manchester United':u_res[0,:],other_team_name:u_res[1,:],'Points':points})
df=df.sort_values(by='Points',ascending=False)
df.head(10)

In [ ]:
season.matches_remaining(team_filter=['Manchester U'])

In [ ]:
ax=None
for team in calib.teams.values():
    if team.name in ['Manchester United','Leicester City','Chelsea']:
    #if team.name in ['BPLHome','BPLAway']:
        ax=team.plt(ax=ax)
        #ax[0].axvline(x=37)
        #ax[1].axvline(x=37)


In [ ]:
df=pd.read_csv('https://projects.fivethirtyeight.com/soccer-api/club/spi_global_rankings.csv')
df=df.set_index('name')
df = df.join(sr).reset_index()[['name','off','def','Off','Deff']].dropna()
df.head(10)

In [ ]:
plt.plot(df['off'],df['Off'],'.')
poly = np.polyfit(df['off'],df['Off'],1)
plt.plot(df['off'],np.polyval(poly,df['off']))
poly

In [ ]:
plt.plot(df['def'],df['Deff'],'.')
poly = np.polyfit(df['def'],df['Deff'],1)
plt.plot(df['def'],np.polyval(poly,df['def']))
poly

In [ ]:
plt.plot(df['off']/df['def'],df['Off']/df['Deff'],'.')